In [4]:
import mysql.connector
import pandas as pd

conexao = mysql.connector.connect(user='root', password='root', host='localhost', database='python')
cursor = conexao.cursor() 

In [5]:
caminho_categorias = r"/Users/juniorsilva/develop/data/estudos-engenharia-dados/engenharia-dados-com-python-e-banco-de-dados/MySQL/4.Arquivos-CSV/categories.csv"

df_categorias = pd.read_csv(caminho_categorias, sep=',')
df_categorias.head()

,category_id,category_name
0,1,Children Bicycles
1,2,Comfort Bicycles
2,3,Cruisers Bicycles
3,4,Cyclocross Bicycles
4,5,Electric Bikes


In [6]:
try:
    # Exclui todos os dados na tabela 'categories' antes de realizar a carga dos novos dados
    cursor.execute("DELETE FROM python.categories")
    conexao.commit()  # Confirma a exclusão dos dados

    # Itera sobre as linhas do DataFrame df_categorias
    for _, df_line in df_categorias.iterrows():
        # Extrai o id e o nome da categoria da linha atual
        category_id = df_line['category_id']
        category_name = df_line['category_name']

        # Verifica se o id da categoria já existe na tabela
        cursor.execute("SELECT category_id FROM python.categories WHERE category_id = %s", (category_id,))

        result = cursor.fetchone() is not None # Se retornar algo, a categoria existe

        if not result:
            # Se a categoria não existir, insere a nova categoria no banco de dados
            comando = """
                INSERT INTO python.categories (category_id, category_name) VALUES (%s, %s)
            """
            valores = (category_id, category_name)
            cursor.execute(comando, valores)  # Executa o comando de inserção
        else:
            # Caso a categoria já exista, imprime uma mensagem e ignora a inserção
            print(f"A categoria com ID {category_id} já existe, ignorando inserção.")
    
    # Se tudo correr bem, confirma a transação, persistindo as mudanças no banco de dados
    conexao.commit()

except Exception as e:
    # Em caso de erro, desfaz todas as operações executadas até o momento
    conexao.rollback()
    # Exibe a mensagem de erro para o diagnóstico
    print(f"Erro durante a inserção: {e}")
finally:
    # Fecha o cursor para liberar o recurso
    cursor.close()
    # Fecha a conexão com o banco de dados
    conexao.close()
